#Author: ALEJANDRO VALLIN ROMO

## Overview
This project generates a comprehensive Date Dimension Table in a CSV format, which includes various attributes for dates ranging from January 1, 2023, to December 31, 2023. The date range can be easily changed to the needs of the used by modifying the variables start_date and end_date. After running the program, a CSV with the name "date_dimension.csv" is generated on the current working directory, and it can be visualized on Excel or PowerBI as a table where each row is a day, and each column is a different characteristic of that day.
The table is designed for use in data warehousing and business intelligence applications for companies that do business both in Mexico and in the United States, so it provides both date formats used in Mexico (DD/MM/YYYY) and in the USA (MM/DD/YYYY), as well providing a list of holidays celebrated in both Mexico and the USA.

## How to Run the Code
Simply click on "Runtime" on the upper left side of Google Colab, and then click "Run All". Alternatively you can do Ctrl + F9.

## Output
"date_dimension.csv" will be generated. to download it go to "Files" on the left hand side of Google Colab (folder symbol) and you will see there the csv file. Click on the three dots and select "Download".

The csv file can be opened with Excel if you like to inspect it, but is not meant to be used as an Excel file.
Here are the first 3 lines of the table as an example of the formatting:

|DateID|Date|DateMex|DateUSA|DatNum|DayName|Weekday|Week|Month|MonthName|Quarter|QuarterName|Year|MonthYear|IsHoliday|HolidayName|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|20230101|2023-01-01|01/01/2023|01/01/2023|1|Sunday|0|52|1|January|1|Q1|2023|Jan-2023|1|New Year's Day|
|20230102|2023-01-02|02/01/2023|01/02/2023|2|Monday|1|1|1|January|1|Q1|2023|Jan-2023|0|NaN|
|20230103|2023-01-03|03/01/2023|01/03/2023|3|Tuesday|1|1|1|January|1|Q1|2023|Jan-2023|0|Nan|

## Future
A few holidays still need to be added, as well as "dynamic" holidays such as Labor day un the USA (The first Monday of September).
I plan to have two tables, one for Holidays in Mexico, and another one for Holidays in the USA. This means that the column "IsHoliday" will have to be divided into something like "IsHolidayMex" and "IsHolidayUSA", and the column "HolidayName" will probably have to be modified in some way as well.

Finally, other additional columns could be added if some unexpected necessities arise after I start to use these dimension tables on PowerBI projects.

## The Code
If you want to modify what the output table contains, here is an explanation in detail of the contents:

### Importing Libraries
Only pandas and numpy are necesary

In [ ]:
import pandas as pd
import numpy as np

### Defining Date Range
Right now the table will have all days between January 1st, 2023 and Dicember 31, 2023. If you prefer a different date range just modify the variables start_date and end_date.

In [ ]:
# Define the date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Create a date range
date_range = pd.date_range(start_date, end_date)

### Creating the Dataframe for the Table
First we create a dataframe called date_dim.
Then we add all the attributes which don't need extra steps to add.

In [ ]:
# Create a DataFrame
date_dim = pd.DataFrame(date_range, columns=['Date'])

# Add attributes
date_dim['DateID'] = date_dim['Date'].dt.strftime('%Y%m%d').astype(int)
date_dim['DateUSA']= date_dim['Date'].dt.strftime('%m/%d/%Y')
date_dim['DateMex']= date_dim['Date'].dt.strftime('%d/%m/%Y')
date_dim['DayNum'] = date_dim['Date'].dt.day
date_dim['DayName'] = date_dim['Date'].dt.day_name()
date_dim['Weekday'] = np.where(date_dim['Date'].dt.weekday < 5, 1, 0)
date_dim['Week'] = date_dim['Date'].dt.isocalendar().week
date_dim['Month'] = date_dim['Date'].dt.month
date_dim['MonthName'] = date_dim['Date'].dt.strftime('%B')
date_dim['Quarter'] = date_dim['Date'].dt.quarter
date_dim['QuarterName'] = 'Q' + date_dim['Quarter'].astype(str)
date_dim['Year'] = date_dim['Date'].dt.year
date_dim['MonthYear']= date_dim['Date'].dt.strftime('%b-%Y')

### Holidays
First we make a table for all the holidays. If there are holidays you wish to remove or add, do it by modyfing object "holidays".

In [ ]:
# Define holidays
holidays = [
    (1,1,"New Year's Day"),
    (9,16,"Mexico's Independence Day"),
    (12,25,"Christmas Day"),
    (5,1,"Labor Day (Mexico)")
]

Then we convert the holidays into a dictionary for easy retrieval.

Then we create a temporary column called "MonthDay" where we extract the month and the day of the holiday.

Then we map the holiday dictionary on the "MonthDay" to create the new "HolidayName" column.

Then we create column "IsHoliday" by finding which days have a HolidayName asigned to them.

And finally we delite the temporary column we created, just to keep things clean.

In [ ]:
# Convert holidays to a dictionary for easier lookup
holiday_dict = {(month, day): name for month, day, name in holidays}

# Extract month and day from the Date column
date_dim['MonthDay'] = list(zip(date_dim['Date'].dt.month, date_dim['Date'].dt.day))

# Check if the MonthDay is in the holiday dictionary and assign holiday names
date_dim['HolidayName'] = date_dim['MonthDay'].map(holiday_dict)

# Mark holidays with a boolean column
date_dim['IsHoliday'] = date_dim['HolidayName'].notna().astype(int)

# Drop the temporary MonthDay column
date_dim = date_dim.drop(columns=['MonthDay'])

Finally we reorder the columns so that it makes a bit more sense, print the first 10 days of the file to make sure the output is what we wanted, and finally save the Dataframe as the CSV file.

In [ ]:
# Reorder columns
date_dim = date_dim[['DateID', 'Date', 'DateMex', 'DateUSA', 'DayNum', 'DayName', 'Weekday', 'Week', 'Month', 'MonthName', 'Quarter', 'QuarterName', 'Year', 'MonthYear', 'IsHoliday', 'HolidayName']]

# Display the DataFrame
print(date_dim.head(10))

# Save to CSV
date_dim.to_csv('date_dimension.csv', index=False)

## Raw Code

```python
import pandas as pd
import numpy as np

# Define the date range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Create a date range
date_range = pd.date_range(start_date, end_date)

# Create a DataFrame
date_dim = pd.DataFrame(date_range, columns=['Date'])

# Add attributes
date_dim['DateID'] = date_dim['Date'].dt.strftime('%Y%m%d').astype(int)
date_dim['DateUSA']= date_dim['Date'].dt.strftime('%m/%d/%Y')
date_dim['DateMex']= date_dim['Date'].dt.strftime('%d/%m/%Y')
date_dim['DayNum'] = date_dim['Date'].dt.day
date_dim['DayName'] = date_dim['Date'].dt.day_name()
date_dim['Weekday'] = np.where(date_dim['Date'].dt.weekday < 5, 1, 0)
date_dim['Week'] = date_dim['Date'].dt.isocalendar().week
date_dim['Month'] = date_dim['Date'].dt.month
date_dim['MonthName'] = date_dim['Date'].dt.strftime('%B')
date_dim['Quarter'] = date_dim['Date'].dt.quarter
date_dim['QuarterName'] = 'Q' + date_dim['Quarter'].astype(str)
date_dim['Year'] = date_dim['Date'].dt.year
date_dim['MonthYear']= date_dim['Date'].dt.strftime('%b-%Y')

# Define holidays
holidays = [
    (1,1,"New Year's Day"),
    (9,16,"Mexico's Independence Day"),
    (12,25,"Christmas Day"),
    (5,1,"Labor Day (Mexico)")
]

# Convert holidays to a dictionary for easier lookup
holiday_dict = {(month, day): name for month, day, name in holidays}

# Extract month and day from the Date column
date_dim['MonthDay'] = list(zip(date_dim['Date'].dt.month, date_dim['Date'].dt.day))

# Check if the MonthDay is in the holiday dictionary and assign holiday names
date_dim['HolidayName'] = date_dim['MonthDay'].map(holiday_dict)

# Mark holidays with a boolean column
date_dim['IsHoliday'] = date_dim['HolidayName'].notna().astype(int)

# Drop the temporary MonthDay column
date_dim = date_dim.drop(columns=['MonthDay'])

# Reorder columns
date_dim = date_dim[['DateID', 'Date', 'DateMex', 'DateUSA', 'DayNum', 'DayName', 'Weekday', 'Week', 'Month', 'MonthName', 'Quarter', 'QuarterName', 'Year', 'MonthYear', 'IsHoliday', 'HolidayName']]

# Display the DataFrame
print(date_dim.head(10))

# Save to CSV
date_dim.to_csv('date_dimension.csv', index=False)
```